<a href="https://colab.research.google.com/github/Agudelo18/UDEA-ai4eng-20252---Pruebas-Saber-Pro-Colombia/blob/main/04%20-%20modelo%20con%20DecisionTreeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.28GB/s]


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:

# 3 - CARGAR LOS DATOS INICIALES
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 4 - EXCLUIR COLUMNAS DE ALTA CARDINALIDAD
train_df = train_df.drop(columns=['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO'])
test_df = test_df.drop(columns=['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO'])

# 5 - APLICAR MAPAS Y CONVERSIONES
# 5.1 - ESTU_VALORMATRICULAUNIVERSIDAD
tuition_average_mapping = {
    "No pagó matrícula": 0.0, "Menos de 500 mil": 0.25, "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75, "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millions et moins de 5.5 millions": 4.75, "Entre 5.5 millones y menos de 7 millones": 6.25, "Más de 7 millones": 7.5
}
train_df['E_VALORMATRICULAUNIVERSIDAD'] = train_df['E_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)
test_df['E_VALORMATRICULAUNIVERSIDAD'] = test_df['E_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

# 5.2 - FAMI_ESTRATOVIVIENDA
housing_strata_mapping = {"Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3, "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6, "Sin Estrato": None}
train_df['F_ESTRATOVIVIENDA'] = train_df['F_ESTRATOVIVIENDA'].map(housing_strata_mapping)
test_df['F_ESTRATOVIVIENDA'] = test_df['F_ESTRATOVIVIENDA'].map(housing_strata_mapping)

# 5.3 - FAMI_TIENEINTERNET et ESTU_PAGOMATRICULAPROPIO
binary_mappings = {"Si": 1, "No": 0}
train_df['F_TIENEINTERNET'] = train_df['F_TIENEINTERNET'].map(binary_mappings)
test_df['F_TIENEINTERNET'] = test_df['F_TIENEINTERNET'].map(binary_mappings)
train_df['E_PAGOMATRICULAPROPIO'] = train_df['E_PAGOMATRICULAPROPIO'].map(binary_mappings)
test_df['E_PAGOMATRICULAPROPIO'] = test_df['E_PAGOMATRICULAPROPIO'].map(binary_mappings)

# 5.4 - Codificación de la variable objetivo
target_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(target_mapping)

# 5.5 - Separación de características y variable objetivo
X_train = train_df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_df['RENDIMIENTO_GLOBAL']
X_test = test_df.drop(columns=['ID'])


# 6 - PROCESO PARA PRETRATAMIENTO
# 6.1 - Identificación de columnas de baja cardinalidad para OneHotEncoder
low_cardinality_cols = [col for col in X_train.columns if X_train[col].nunique() < 10 and X_train[col].dtype == "object"]

# 6.2 - Proceso de pretratamiento simplificada
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), X_train.select_dtypes(include=['float64', 'int64']).columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), low_cardinality_cols)
    ]
)

# 6.3 - Proceso de entrenamiento con un modelo de árbol de decisión sin hiperparámetros
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

# 7 - ENTRENAR EL MODELO
pipeline.fit(X_train, y_train)

# 8 - HACER PREDICCIONES SOBRE EL CONJUNTO DE DATOS test
predictions = pipeline.predict(X_test)

# 9 - CONVERTIR LAS PREDICCIONES EN LA REDACCIÓN ORIGINAL
target_inverse_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
predictions_labels = [target_inverse_mapping[pred] for pred in predictions]

# 10 - CREA EL ARCHIVO DE PRESENTACIÓN, y asegúrate de que las dimensiones son consistentes con las requeridas por la competición de Kaggle.
submission_df = pd.DataFrame({
    "ID": test_df['ID'],
    "RENDIMIENTO_GLOBAL": predictions_labels
})
submission_df.to_csv("submission.csv", index=False)

print("Dimensiones de la clasificación submission.csv (filas, columnas) ", submission_df.shape)


Dimensiones de la clasificación submission.csv (filas, columnas)  (296786, 2)
